In [1]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
from tensorflow.keras.layers import Dense, MaxPool2D, Conv2D, Flatten
from tensorflow.keras.optimizers import SGD, Adam

In [2]:
folder_address = '/content/drive/MyDrive/Datasets/Flowers/'

In [3]:
width = height = 224
batch_size = 8

In [4]:
data_generator = ImageDataGenerator(
    rescale = 1 / 255,
    validation_split = 0.2,
)

train_data = data_generator.flow_from_directory(folder_address + 'Train',
                                                target_size = (width, height),
                                                class_mode = 'categorical',
                                                batch_size = batch_size,
                                                shuffle = True,
                                                subset='training')

val_data = data_generator.flow_from_directory(folder_address + 'Train',
                                              target_size = (width, height),
                                              class_mode = 'categorical',
                                              batch_size = batch_size,
                                              shuffle = True,
                                              subset='validation')

test_data = data_generator.flow_from_directory(folder_address + 'Test',
                                              target_size = (width, height),
                                              class_mode = 'categorical',
                                              batch_size = batch_size,
                                              shuffle = True)

Found 884 images belonging to 17 classes.
Found 204 images belonging to 17 classes.
Found 272 images belonging to 17 classes.


In [5]:
base_model = tf.keras.applications.ResNet50V2(
    input_shape=(width, height, 3),
    include_top = False,
    weights = 'imagenet',
    pooling = 'max'
    )

94683136/94668760 [==============================] - 1s 0us/step


In [6]:
base_model.summary()

Model: "resnet50v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
_________________________________________________________________________________________

In [7]:
model = tf.keras.models.Sequential([
    base_model,
    Dense(17, activation='softmax')                          
])

In [8]:
model.compile(optimizer = tf.keras.optimizers.SGD(),
              loss = tf.keras.losses.binary_crossentropy,
              metrics='accuracy')

In [11]:
model.fit(train_data,
          steps_per_epoch = train_data.samples / batch_size,
          validation_data = val_data,
          validation_steps = val_data.samples / batch_size,
          epochs = 15
)

Epoch 1/15
110/110 [==============================] - 27s 241ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.0566 - val_accuracy: 0.9118
Epoch 2/15
110/110 [==============================] - 26s 239ms/step - loss: 0.0029 - accuracy: 0.9966 - val_loss: 0.0525 - val_accuracy: 0.9118
Epoch 3/15
110/110 [==============================] - 27s 240ms/step - loss: 0.0037 - accuracy: 0.9989 - val_loss: 0.0582 - val_accuracy: 0.8971
Epoch 4/15
110/110 [==============================] - 26s 239ms/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.0555 - val_accuracy: 0.9216
Epoch 5/15
110/110 [==============================] - 26s 238ms/step - loss: 0.0027 - accuracy: 0.9989 - val_loss: 0.0577 - val_accuracy: 0.9118
Epoch 6/15
110/110 [==============================] - 26s 239ms/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 0.0538 - val_accuracy: 0.8971
Epoch 7/15
110/110 [==============================] - 26s 239ms/step - loss: 0.0018 - accuracy: 0.9989 - val_loss: 0.0553 - val_ac

In [12]:
model.evaluate(test_data)

34/34 [==============================] - 130s 4s/step - loss: 0.1229 - accuracy: 0.7868


[0.12288638949394226, 0.7867646813392639]